In [1]:
import pickle
import numpy as np 
import pandas as pd 
import math

import sys
sys.path.append('../utils')
from treasury_base import *
from scipy.stats import t
from sklearn.metrics import mean_squared_error as mse

In [27]:
cross_val_window = 20
with open(f"pickle/dfs_{cross_val_window}_steps_ahead.pkl", "rb") as file:
    loaded_dataframes = pickle.load(file)

# Extract individual DataFrames
naive_df = loaded_dataframes["naive_df"]
kan_df = loaded_dataframes["kan_df"]
truth_df = loaded_dataframes["truth_df"]

# Use the DataFrames
kan_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,4.709641,4.742804,4.658519,4.543219,4.253460,4.228101,4.157092,4.109575,4.220700,4.387587,4.643235,4.528277
1,4.698521,4.739191,4.652665,4.548754,4.251203,4.230456,4.146784,4.093434,4.195589,4.368830,4.625434,4.505712
2,4.696217,4.737129,4.648003,4.556075,4.248187,4.235888,4.138464,4.077472,4.172333,4.353811,4.608896,4.483943
3,4.697104,4.735684,4.647111,4.570990,4.252269,4.246329,4.138485,4.066039,4.151629,4.341901,4.596337,4.463836
4,4.698404,4.734125,4.643395,4.581351,4.249566,4.253631,4.133756,4.050708,4.127117,4.325795,4.580877,4.443470


In [51]:
cols = truth_df.columns
mode = 'two-sided'
cross_val_list = [20]

result = dict()
for i in range(12):
    result[i] = list()

for cross_val_window in cross_val_list:
    with open(f"pickle/dfs_{cross_val_window}_steps_ahead.pkl", "rb") as file:
        loaded_dataframes = pickle.load(file)

    # Extract individual DataFrames
    naive_df = loaded_dataframes["naive_df"]
    kan_df = loaded_dataframes["kan_df"]
    truth_df = loaded_dataframes["truth_df"]

    for i in range(12):
        y1 = kan_df.iloc[:, i].values
        y2 = naive_df.iloc[:, i].values
        y = truth_df.iloc[:, i].values

        # print(f'{cols[i]}: {dm_test(y, y1, y2, h=1, mode=mode)}')
        if mse(y1, y) < mse(y2, y):
            result[i].append(True)
        else:
            result[i].append(False)

out = pd.DataFrame(result)
out.columns = cols
out.index = cross_val_list
out


,1 Mo,2 Mo,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
20,False,False,False,False,False,False,False,False,False,False,False,False


In [52]:
print(mse(kan_df.values, truth_df.values))
print(mse(naive_df.values, truth_df.values))

0.21259998215473952
0.025231111111111093


In [38]:
print(mse(kan_df.values, truth_df.values))
print(mse(naive_df.values, truth_df.values))

0.013525092183985113
0.002106250000000003


,0,1,2,3,4,5,6,7,8,9,10,11
0,4.675464,4.690526,4.634253,4.497424,4.335040,4.255333,4.190395,4.219443,4.259672,4.368931,4.652386,4.555999
1,4.655106,4.691010,4.634375,4.487466,4.326359,4.255643,4.185925,4.189279,4.223151,4.351579,4.632552,4.546283
2,4.619245,4.686298,4.630958,4.484269,4.310511,4.254398,4.193182,4.188441,4.219110,4.353633,4.625464,4.549663
3,4.574238,4.676403,4.622022,4.489915,4.321399,4.268562,4.215420,4.207200,4.246131,4.370638,4.636329,4.561916
4,4.526525,4.674935,4.633108,4.487393,4.314217,4.263392,4.229566,4.162215,4.185568,4.348195,4.578426,4.554968
5,4.473090,4.672672,4.623696,4.476923,4.286868,4.253886,4.246528,4.106812,4.125637,4.310724,4.517031,4.516785
6,4.502060,4.680983,4.609286,4.478089,4.281439,4.253975,4.253625,4.128751,4.149595,4.340913,4.513011,4.485161
7,4.565794,4.621794,4.544572,4.428077,4.196578,4.198806,4.211650,4.091491,4.115646,4.308330,4.462101,4.433904
8,4.588640,4.599369,4.509853,4.414566,4.224351,4.270405,4.205554,4.079066,4.095842,4.268443,4.437900,4.404545
9,4.422199,4.519417,4.469549,4.403694,4.216850,4.286607,4.188829,4.108359,4.123902,4.331592,4.499396,4.490396
